In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
%cd drive/MyDrive/mistral

Mounted at /content/drive
/content/drive/MyDrive/mistral


In [2]:
!pip -q install git+https://github.com/huggingface/transformers # need to install from github
!pip -q install bitsandbytes accelerate xformers einops
!pip install datasets

import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM

from datasets import load_dataset
raw_datasets = load_dataset("orkg/SciQA")
print(raw_datasets)

model_id = "mistralai/Mistral-7B-Instruct-v0.1"
bnb_config = transformers.BitsAndBytesConfig(
load_in_4bit=True,
bnb_4bit_use_double_quant=True,
bnb_4bit_quant_type="nf4",
bnb_4bit_compute_dtype=torch.bfloat16
)

model = transformers.AutoModelForCausalLM.from_pretrained(
model_id,
trust_remote_code=True,
quantization_config=bnb_config,
device_map='auto',
)

tokenizer = transformers.AutoTokenizer.from_pretrained(
model_id,
)

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 30.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 MB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.2/670.2 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 64.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 67.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 51.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 809.8 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Generating train split:   0%|          | 0/1795 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/257 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/513 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'query_type', 'question', 'paraphrased_question', 'query', 'template_id', 'query_shape', 'query_class', 'auto_generated', 'number_of_patterns'],
        num_rows: 1795
    })
    validation: Dataset({
        features: ['id', 'query_type', 'question', 'paraphrased_question', 'query', 'template_id', 'query_shape', 'query_class', 'auto_generated', 'number_of_patterns'],
        num_rows: 257
    })
    test: Dataset({
        features: ['id', 'query_type', 'question', 'paraphrased_question', 'query', 'template_id', 'query_shape', 'query_class', 'auto_generated', 'number_of_patterns'],
        num_rows: 513
    })
})


...

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

In [3]:
import json

def save_json(filename,data):
    with open(filename, "w", encoding="utf-8") as json_file:
        print(json.dumps(data), file=json_file)


def load_json(file__name):
    try:
        data_file = open(file__name, "r", encoding='utf-8')
        file_data = json.loads(data_file.read())
        data_file.close()
        return file_data
    except FileNotFoundError:
        return None


def main(filename="test_a"):
    data = load_json(filename)
    # print(data)
    if data is None:
        return
    else:
        query_list = data["questions"]
        suggestions = data["suggestions"]
        gs = data["generated_sparql"]
        # lens = data["prompt_len"]
        sparql = data["sparql"]

    print(len(query_list))

    q_list = query_list

    for question in q_list[len(gs):]:
        print(len(gs))
        encodeds = tokenizer(question, return_tensors="pt", add_special_tokens=False)
        model_inputs = encodeds.to('cuda')

        generated_ids = model.generate(**model_inputs, max_new_tokens=1000, do_sample=True)
        decoded = tokenizer.batch_decode(generated_ids)
        print(decoded[0])

        gs.append(decoded[0])
        result = {"questions": query_list, "sparql": sparql, "generated_sparql": gs, "suggestions": suggestions}
        save_json(filename, result)

In [4]:
main("mistral_ddp_data.json")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


513
0


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> [INST]input (English text): Can you provide the highest benchmark result, including the metric and score, for the Story Cloze Test dataset?
 output (Sparql query): [/INST]SELECT DISTINCT ?metric ?metric_lbl (MAX( ?value) AS ?score) WHERE { { SELECT ?metric ?metric_lbl ?value WHERE { ?dataset a orkgc:Dataset; rdfs:label ?dataset_lbl. FILTER (str( ?dataset_lbl) = "Story Cloze Test") ?benchmark orkgp:HAS_DATASET ?dataset; orkgp:HAS_EVALUATION ?eval. ?eval orkgp:HAS_VALUE ?value. OPTIONAL { ?eval orkgp:HAS_METRIC ?metric. ?metric rdfs:label ?metric_lbl. } ?cont orkgp:HAS_BENCHMARK ?benchmark. OPTIONAL { ?cont orkgp:HAS_MODEL ?model. ?model rdfs:label ?model_lbl. } } ORDER BY DESC( ?value) } } GROUP BY ?metric ?metric_lbl</s><s> [INST]input (English text): Which countries are considered in the papers about geopolitics?
 output (Sparql query): [/INST]SELECT DISTINCT ?location WHERE { ?_ orkgp:compareContribution [ orkgp:P32 [ rdfs:label ?label ]; orkgp:P5049 ?location ] FILTER(REGEX(STR(

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


...

<s> [INST]input (English text): Which model has achieved the highest Best Score score on the Atari 2600 Q*Bert benchmark dataset?
 output (Sparql query): [/INST]SELECT DISTINCT ?model ?model_lbl WHERE { ?metric a orkgc:Metric; rdfs:label ?metric_lbl. FILTER (str( ?metric_lbl) = "Best Score") { SELECT ?model ?model_lbl WHERE { ?dataset a orkgc:Dataset; rdfs:label ?dataset_lbl. FILTER (str( ?dataset_lbl) = "Atari 2600 Q*Bert") ?benchmark orkgp:HAS_DATASET ?dataset; orkgp:HAS_EVALUATION ?eval. ?eval orkgp:HAS_VALUE ?value; orkgp:HAS_METRIC ?metric. ?cont orkgp:HAS_BENCHMARK ?benchmark; orkgp:HAS_MODEL ?model. ?model rdfs:label ?model_lbl. } ORDER BY DESC( ?value) LIMIT 1 } } </s><s> [INST]input (English text): Which countries are considered in the papers about geopolitics?
 output (Sparql query): [/INST]SELECT DISTINCT ?location WHERE { ?_ orkgp:compareContribution [ orkgp:P32 [ rdfs:label ?label ]; orkgp:P5049 ?location ] FILTER(REGEX(STR( ?label), "geopoli ?tics")) } </s><s> [INST]input

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> [INST]input (English text): Can you provide the highest benchmark result, including the metric and score, for the One Billion Word dataset?
 output (Sparql query): [/INST]SELECT DISTINCT ?metric ?metric_lbl (MAX( ?value) AS ?score) WHERE { { SELECT ?metric ?metric_lbl ?value WHERE { ?dataset a orkgc:Dataset; rdfs:label ?dataset_lbl. FILTER (str( ?dataset_lbl) = "One Billion Word") ?benchmark orkgp:HAS_DATASET ?dataset; orkgp:HAS_EVALUATION ?eval. ?eval orkgp:HAS_VALUE ?value. OPTIONAL { ?eval orkgp:HAS_METRIC ?metric. ?metric rdfs:label ?metric_lbl. } ?cont orkgp:HAS_BENCHMARK ?benchmark. OPTIONAL { ?cont orkgp:HAS_MODEL ?model. ?model rdfs:label ?model_lbl. } } ORDER BY DESC( ?value) } } GROUP BY ?metric ?metric_lbl</s><s> [INST]input (English text): Which countries are considered in the papers about geopolitics?
 output (Sparql query): [/INST]SELECT DISTINCT ?location WHERE { ?_ orkgp:compareContribution [ orkgp:P32 [ rdfs:label ?label ]; orkgp:P5049 ?location ] FILTER(REGEX(STR(

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> [INST]input (English text): List the datasets benchmarked under the Skills Assessment research problem?
 output (Sparql query): [/INST]SELECT DISTINCT ?dataset ?dataset_lbl WHERE { ?problem a orkgc:Problem; rdfs:label ?problem_lbl. FILTER (str( ?problem_lbl) = "Skills Assessment") ?dataset a orkgc:Dataset; rdfs:label ?dataset_lbl. ?benchmark orkgp:HAS_DATASET ?dataset. ?cont orkgp:HAS_BENCHMARK ?benchmark; orkgp:P32 ?problem. } </s><s> [INST]input (English text): Which vegetables are utilized for betanin extraction?
 output (Sparql query): [/INST]SELECT ?vegetables, ?vegetables_labels WHERE { orkgr:R75363 orkgp:compareContribution ?contrib. ?contrib orkgp:P35147 ?compounds. ?compounds rdfs:label ?compounds_labels. FILTER(REGEX( ?compounds_labels, "etanin")) ?contrib orkgp:P35148 ?vegetables. ?vegetables rdfs:label ?vegetables_labels. } </s><s> [INST]input (English text): What is the scope of "Decentralised Authoring, Annotations and Notifications for a Read-Write Web with Dokieli"?